In [1]:
import pandas as pd
from sqlalchemy import create_engine
import dask.dataframe as dd

In [2]:
%load_ext blackcellmagic

In [3]:
engine = create_engine("sqlite:///tmp/db.sqlite")

In [9]:
fund_type = pd.read_excel("tmp/Fund Type.xlsx")
fund_type.head()

,TrustFund Num,TrustKind
0,5129978,NaN
1,5106562,NaN
2,5108519,NaN
3,5116629,NaN
4,5128616,NaN


In [10]:
fund_type.TrustKind.value_counts()

מחקה    936
Name: TrustKind, dtype: int64

In [11]:
fund_type.TrustKind.isna().sum()

1180

In [12]:
copy_funds = fund_type.set_index("TrustFund Num").rename_axis("fund").dropna().index.to_series()
copy_funds.to_csv("tmp/copy_funds.csv", index=False, header=True)

In [13]:
copy_funds.index

Int64Index([5123351, 5127378, 5128467, 5127451, 5127436, 5128459, 5128624,
            5127410, 5125703, 5129648,
            ...
            1144823, 1144690, 1144070, 1145184, 1144344, 1144351, 1144369,
            1145259, 1144260, 1145036],
           dtype='int64', name='fund', length=936)

In [22]:
original = pd.read_sql("""
select "Fund Num" fund, "Manager Num" manager, "Instrument Num" instrument, "Fair Value" value, DateCode date, "Row Num" row 
from k203 where "Instrument Type Num" = 'מניה'
""", engine)

In [23]:
df = original.drop_duplicates()
df["date"] = pd.to_datetime(df.date)

c:\users\imri\github\open_pension\data_science\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
df.head()

,manager,instrument,date,value
0,510791031,1080324,2017-11-30,594045.64
1,510791031,1080324,2017-12-31,621811.50
2,510791031,1080324,2018-01-31,742410.00
3,510791031,1080324,2018-02-28,725666.51
4,510791031,1080324,2018-03-29,593455.92


In [25]:
df = df[df.fund.isin(copy_funds.index)]

In [26]:
df = dd.from_pandas(df, npartitions=8).groupby(["manager", "instrument", "date"])["value"].aggregate("sum").reset_index().compute()

In [27]:
df.to_csv("tmp/dataset.csv")

In [8]:
prices = pd.read_excel("tmp/Stock Pice.xlsx")

In [11]:
prices.columns

Index(['Dim Date.Date', 'Instrument Num', 'Price', 'Registreted Capital'], dtype='object')

In [13]:
prices = prices.rename(columns={
    "Dim Date.Date": "date",
    "Instrument Num": "instrument",
    "Price": "price"
})

In [15]:
prices = prices.drop(columns=['Registreted Capital'])

In [38]:
prices.instrument = prices.instrument.astype(str)

In [39]:
prices.to_csv("tmp/prices.csv")

In [30]:
indices = pd.read_excel("tmp/Indexs.xlsx")

In [31]:
indices.head()

,Date,Index,Symbol,Instrument Nom,ISIN,Weight
0,2017-11-30,"ת""א 35",ביג,1097260,IL0010972607,0.40672
1,2017-11-30,"ת""א 35",סלקום,1101534,IL0011015349,0.73235
2,2017-11-30,"ת""א 35",דלק קבוצה,1084128,IL0010841281,0.77757
3,2017-11-30,"ת""א 35",חברה לישראל,576017,IL0005760173,0.94221
4,2017-11-30,"ת""א 35",פרטנר,1083484,IL0010834849,0.96408


In [32]:
indices = indices.rename(columns=lambda x: x.replace(" ", "_").lower())

In [33]:
indices.columns

Index(['date', 'index', 'symbol', 'instrument_nom', 'isin', 'weight'], dtype='object')

In [34]:
indices = indices.rename(
    columns={"instrument_nom": "instrument", "index": "index_name"}
)

In [35]:
indices = indices[['date', 'index_name', 'instrument', 'weight']]

In [37]:
indices.index_name = indices.index_name.apply(lambda x: x.replace('"', ""))

In [38]:
indices.to_csv("tmp/indices.csv")